In [2]:
from langchain.document_loaders import TextLoader 
from langchain.document_loaders import UnstructuredURLLoader #for unstructured text i.e html page to extract text

In [3]:
loader = UnstructuredURLLoader(urls = [
   ' https://exputer.com/exputer/metal-gear-solid-3-highly-faithful-remake/'
])

In [4]:
data = loader.load()
len(data)

1

In [5]:
data[0].page_content

"Menu\n\nSwitch skin\n\nNews\n\n\tExclusives\n\tIndustry\n\tGames\n\tPatents\n\nOriginals\n\nInterviews\n\nGuides\n\n\tError Fixes\n\tGame Settings\n\tTier Lists\n\tGame Codes\n\tUpcoming Games\n\tPalworld\n\tFortnite\n\tStarfield\n\tRemnant 2\n\tBaldur’s Gate 3\n\tDiablo 4\n\tElden Ring\n\nReviews\n\nGaming Hub\n\n\tVideo Game Recommendation Engine\n\tFallout 4 Calculator\n\tQuiz\n\t\n\t\tGuess The Game By Its Logo\n\t\tGeneral Knowledge Quiz\n\t\tElden Ring Quiz\n\t\tGTA V Quiz\n\t\tRDR2 Quiz\n\t\tMinecraft Quiz\n\t\tThe Last Of Us Quiz\n\nForums\n\nMore\n\n\tVideos\n\tShop\n\tCoaching\n\tWork With Us\n\tDiscord\n\tPatreon\n\nSearch for\n\nSwitch skin\n\nCommunity\n\t\t\t\n\t\t\tFacebookTwitterLinkedInYouTubeInstagramTripAdvisorTikTokRSS\n\nHome\n\neXputer Originals\n\nMetal Gear Solid 3 Remake Shows Incredible Commitment To The Original\n\nMetal Gear Solid 3 Remake Shows Incredible Commitment To The Original\n\nFrom entire sequences to the smallest of details, the dedication is insa

In [6]:
data[0].metadata

{'source': ' https://exputer.com/exputer/metal-gear-solid-3-highly-faithful-remake/'}

In [7]:
#now we divide into several chunks, get merge relevant chunk closer to token size 

In [8]:
from langchain.text_splitter import CharacterTextSplitter

In [9]:
# seperate text on the basis of only one seperator which can lead to chunks greater than specified chunk size
text = """An armed conflict between Israel and Hamas-led Palestinian militant groups[ab] has been taking place chiefly in the Gaza Strip since 7 October 2023. Clashes have also occurred in the Israeli-occupied West Bank and with Hezbollah along the Israel–Lebanon–Golan Heights border. The fifth war of the Gaza–Israel conflict since 2008, it is part of the broader Israeli–Palestinian conflict, and the most significant military engagement in the region since the Yom Kippur War 50 years earlier.[108]
The war began when Hamas-led militant groups launched a surprise attack on Israel on 7 October. An estimated 3,000 militants breached the Gaza–Israel barrier and attacked Israeli civilian communities and military bases. Several thousand rockets were concurrently launched into Israel. During this attack, 1,139 Israelis and foreign nationals were killed, including 766 civilians and 373 security personnel.[ac] In addition, 251 Israelis and foreigners were taken captive into Gaza.[109] Hamas said its attack was in response to the continued Israeli occupation of the Palestinian territories, the blockade of the Gaza Strip, the expansion of illegal Israeli settlements, as well as alleged threats to the Al-Aqsa Mosque and the plight of Palestinian refugees and prisoners.[110][111][112][113][114]
After clearing militants from its territory, Israel responded by launching one of the most destructive bombing campaigns in modern history,[115][116] before commencing a large-scale ground invasion on 27 October with the stated objectives of destroying Hamas and releasing the hostages.[117][118] Since the start of the Israeli operation, more than 35,000 Palestinians in Gaza have been killed,[119] of which the UN estimates at least 56% are women and children.[120] 10,000 others are missing and presumed trapped under rubble.[37] The war has been the deadliest for Palestinians in the entire Arab–Israeli conflict.[121] Nearly all of the strip's 2.3 million Palestinian population has been forcibly displaced,[122][123] and over one hundred thousand Israelis remain internally displaced"""


splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 200,
    chunk_overlap = 0
)
chunks = splitter.split_text(text)
for chunk in chunks:
    print(len(chunk))
    print(chunk)

Created a chunk of size 492, which is longer than the specified 200
Created a chunk of size 798, which is longer than the specified 200


492
An armed conflict between Israel and Hamas-led Palestinian militant groups[ab] has been taking place chiefly in the Gaza Strip since 7 October 2023. Clashes have also occurred in the Israeli-occupied West Bank and with Hezbollah along the Israel–Lebanon–Golan Heights border. The fifth war of the Gaza–Israel conflict since 2008, it is part of the broader Israeli–Palestinian conflict, and the most significant military engagement in the region since the Yom Kippur War 50 years earlier.[108]
798
The war began when Hamas-led militant groups launched a surprise attack on Israel on 7 October. An estimated 3,000 militants breached the Gaza–Israel barrier and attacked Israeli civilian communities and military bases. Several thousand rockets were concurrently launched into Israel. During this attack, 1,139 Israelis and foreign nationals were killed, including 766 civilians and 373 security personnel.[ac] In addition, 251 Israelis and foreigners were taken captive into Gaza.[109] Hamas said i

In [10]:
#we use recursive text splitter that can use multiple seperator
from langchain.text_splitter import RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n",".","?"],
    chunk_size = 200,
    chunk_overlap = 0
)
chunks = r_splitter.split_text(text)
for chunk in chunks:
    print(len(chunk))

147
127
212
6
94
189
197
292
26
286
176
189
139


In [11]:
import os
import streamlit as st
import pickle
import time
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


In [12]:
os.environ['OPENAI_API_KEY'] = ''

In [13]:
llm = OpenAI(temperature = 0.9, max_tokens = 500)
loader = UnstructuredURLLoader(urls = [
   ' https://exputer.com/exputer/metal-gear-solid-3-highly-faithful-remake/'
])
DATA = loader.load()

rec_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
docs = rec_splitter.split_documents(data)
len(docs)

C:\Users\Lenovo\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


15

In [ ]:
embeddings = OpenAIEmbeddings()
vector_index = FAISS.from_documents(docs,embeddings)

In [ ]:
file_path = "vector_index.pkl"
with open(file_path,"wb")as f:
    pickle.dump(vector_index,f)

In [ ]:
if os.path.exists(file_path):
    with open(filepath,"rb") as f:
        vectoridx = pickle.load(f)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever= vectoridx.as_retriever())
query = "what is the gameplay style of metal gear solid 3"
langchain.debug= True
chain({"question":Query},return_only_outputs=True)
